In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import nfl_data_py as nfl
import re
import os
from dotenv import load_dotenv

load_dotenv()

# 01 - Data Gathering
Collects and combines NFL draft data, college football statistics, and rookie season performance data to build a master dataset for fantasy football prediction.

**Data Sources:**
- NFL Draft picks (2019-2024) from Pro Football Reference
- College football stats from the CFBD API
- Rookie NFL season stats from `nfl_data_py`

**Output:** `data/processed/df_master.csv`

## Recommended data fields (per player)
- Identity: `player_id`, `name`, `position`, `draft_year`
- Draft capital: `draft_round`, `draft_pick_overall`, `team`
- College production (final season + career): rushing/receiving/passing as appropriate
- Context: `conference`, `age_on_draft_day`, `height_in`, `weight_lb`, `forty_yd`, etc.
- NFL rookie outcomes (for training): rushing/receiving/passing/fumbles/INT, games played

In [46]:
COLUMN_NAMES = ["player_id", "name", "position", "team", "draft_year", "draft_round", "draft_pick_overall", "age_on_draft_day", "conference", "conference_strength", "height_in", "weight_lb", "forty_yd\nC_Games", "C_Rec", "C_RecYds", "C_RecTD", "C_RushAtt", "C_RushYds", "C_RushTD", "C_PassAtt", "C_PassYds", "C_PassTD", "C_INT\nCar_Games", "Car_Rec", "Car_RecYds", "Car_RecTD", "Car_RushAtt", "Car_RushYds", "Car_RushTD", "Car_PassAtt", "Car_PassYds", "Car_PassTD", "Car_INT\nTeam_RecYds", "Team_RecTD", "Team_RushYds", "Team_RushTD", "Team_PassYds", "Team_PassTD\nRec", "RecYds", "RecTD", "RushYds", "RushTD", "PassYds", "PassTD", "Fumbles", "INT"]
df = pd.DataFrame(columns=COLUMN_NAMES)
print(df)

Empty DataFrame
Columns: [player_id, name, position, team, draft_year, draft_round, draft_pick_overall, age_on_draft_day, conference, conference_strength, height_in, weight_lb, forty_yd
C_Games, C_Rec, C_RecYds, C_RecTD, C_RushAtt, C_RushYds, C_RushTD, C_PassAtt, C_PassYds, C_PassTD, C_INT
Car_Games, Car_Rec, Car_RecYds, Car_RecTD, Car_RushAtt, Car_RushYds, Car_RushTD, Car_PassAtt, Car_PassYds, Car_PassTD, Car_INT
Team_RecYds, Team_RecTD, Team_RushYds, Team_RushTD, Team_PassYds, Team_PassTD
Rec, RecYds, RecTD, RushYds, RushTD, PassYds, PassTD, Fumbles, INT]
Index: []

[0 rows x 46 columns]


## Get Draft Data 
Download Excel files for the drafts of the last 5 years from https://www.pro-football-reference.com/

These are the fields of the above datasets:
- Rnd -- Round selected in draft
- Supplemental draft round indicated with 'S'
- Pick -- Overall Selection in Draft
- Pos -- Position
- Age -- Age as of September 1 of the draft year
- To -- Last Year
- Misc
- AP1 -- First-team all-pro selections
- PB -- Pro Bowl Selections
- St -- Number of years as primary starter for his team at his position
- Approx Val
- wAV -- Weighted Career Approximate Value.
- See the glossary for details on how we compute Approximate Value.
- The career AV is computed by summing
- 100 percent of the AV of his best season,
- 95 percent of the AV of his next-best season,
- 90 percent of the AV of his third-best season,
- and so on
- DrAV -- Weighted AV accumulated for team that drafted this player
- Games
- G -- Games played
#### Passing
- Cmp -- Passes completed
- Att -- Passes attempted
- Yds -- Yards Gained by Passing (For teams, sack yardage is deducted from this total)
- TD -- Passing Touchdowns
- Int -- Interceptions thrown
#### Rushing
- Att -- Rushing Attempts (sacks not included in NFL)
- Yds -- Rushing Yards Gained (sack yardage is not included by NFL)
- TD -- Rushing Touchdowns
#### Receiving
- Rec -- Receptions
- Yds -- Receiving Yards
- TD -- Receiving Touchdowns
#### Tackles
- Solo -- Tackles
- Before 1994: unofficial and inconsistently recorded from team to team. For amusement only.
- 1994-now: unofficial but consistently recorded.
#### Def Interceptions
- Int -- Passes intercepted on defense
- Sk -- Sacks (official since 1982, based on play-by-play, game film, and other research since 1960)

In [ ]:
# Load draft data for each year and tag with draft_year
draft_files = {
    2019: "data/raw/2019_draft.csv",
    2020: "data/raw/2020_draft.csv",
    2021: "data/raw/2021.csv",
    2022: "data/raw/2022_draft.csv",
    2023: "data/raw/2023_draft.csv",
    2024: "data/raw/2024_draft.csv",
}

draft_dfs = []
for year, path in draft_files.items():
    df_year = pd.read_csv(path)
    df_year["draft_year"] = year
    draft_dfs.append(df_year)

drafts_df = pd.concat(draft_dfs, axis=0)

# Build URL-friendly player names for web scraping
for name in drafts_df["Player"]:
    if isinstance(name, str):
        clean_name = re.sub(r'[^\w\s-]', '', name.lower())
        drafts_df.loc[drafts_df["Player"] == name, "url_name"] = clean_name.replace(" ", "-")

print(f"Total draft picks loaded: {len(drafts_df)}")
print(drafts_df.head())

In [63]:
# Filter for relevant positions
positions = ['QB', 'TE', 'WR', 'RB']

drafts_df = drafts_df[drafts_df["Pos"].isin(positions)]
print(len(drafts_df))

314


In [64]:
# Add draft info to master df 
df["name"], df["position"], df["team"], df["draft_year"], df["draft_round"], df["draft_pick_overall"], df["age_on_draft_day"] = drafts_df["Player"], drafts_df["Pos"], drafts_df["Tm"], drafts_df["draft_year"], drafts_df["Rnd"], drafts_df["Pick"], drafts_df["Age"]
print(df[:2])

df["url_name"] = drafts_df["url_name"] # MAKE SURE ORDER and length OF NAMES IS THE SAME!


  player_id            name position team  draft_year  draft_round  \
0       NaN      Joe Burrow       QB  CIN        2020          1.0   
4       NaN  Tua Tagovailoa       QB  MIA        2020          1.0   

   draft_pick_overall  age_on_draft_day conference conference_strength  ...  \
0                 1.0              23.0        NaN                 NaN  ...   
4                 5.0              22.0        NaN                 NaN  ...   

  Team_PassYds Team_PassTD\nRec RecYds RecTD RushYds RushTD PassYds PassTD  \
0          NaN              NaN    NaN   NaN     NaN    NaN     NaN    NaN   
4          NaN              NaN    NaN   NaN     NaN    NaN     NaN    NaN   

  Fumbles  INT  
0     NaN  NaN  
4     NaN  NaN  

[2 rows x 46 columns]


## College Football Stats

In [1]:
def get_player_stats(player_names):
    all_players_data = {}
    for name in player_names:
        found = False 
        for page in range(1,10): 
            url = f"https://www.sports-reference.com/cfb/players/{name}-{page}.html"
            try:
                response = requests.get(url)
                if response.status_code == 200:
                    print(f"Found data for {name} at {url}")
                    soup = BeautifulSoup(response.text, "html.parser")

                    # Get tables 
                    tables = pd.read_html(response.text)

                    # There are mulitple tables per page, map them to their html ids
                    table_map = {}
                    for i, table in enumerate(soup.find_all("table")):
                        table_id = table.get("id", f"table_{i}")
                        table_map[table_id] = tables[i]

                    # Filter only the tables we care about
                    relevant_ids = ["passing_standard", "receiving_standard", "punt_return_standard"]
                    player_tables = {tid: df for tid, df in table_map.items() if tid in relevant_ids}

                    all_players_data[name] = player_tables
                

                    found = True
                    break
            except Exception as e:
                print(f"Error fetching {url}: {e}")
        if not found:
            print(f"No page found for {name}")

    return all_players_data 

In [ ]:
# Test the scraper with a single player
college_stats = get_player_stats(["joe-burrow"])

## Get Rookie Season Data: Method 2 - Using NFL_data_py package
https://pypi.org/project/nfl-data-py/

option: 
nfl.import_seasonal_data(years, s_type)
Returns seasonal data, including various calculated market share stats specific to receivers

years (List[int]) : required, list of years to pull data for (earliest available is 1999)

s_type (str) : optional (default 'REG') season type to include in average ('ALL','REG','POST')


nfl.import_seasonal_pfr(s_type, years)
Returns a dataframe of season-aggregated data sourced from players' pages on pro-football-reference.com. E.g. Patrick Mahomes

s_type (str) : required, the type of stat data to request. Must be one of pass, rec, or rush.

years (List[int]) : optional, years to return data for

In [65]:
def get_player_id_map(names: list[str], years: list[int]) -> dict:
    """
    Build a mapping of player_id -> 'First Last' given player names and corresponding years.
    
    """
    if len(names) != len(years):
        raise ValueError("names and years must have the same length")
    
    player_map = {}
    
    # Loop through each (name, year) pair
    for name, year in zip(names, years):
        roster = nfl.import_seasonal_rosters([year])
        #roster["player_name"] = roster["first_name"] + " " + roster["last_name"]
        
        match = roster[roster["player_name"] == name]
        
        if not match.empty:
            player_id = match.iloc[0]["player_id"]
            player_map[player_id] = {
                "name": name,
                "rookie_year": year,
                "position": match.iloc[0]["position"],
                "team": match.iloc[0]["team"],
                "height_in": match.iloc[0].get("height_in"),
                "weight_lb": match.iloc[0].get("weight_lb"),
                "age_on_draft_day": match.iloc[0].get("age_on_draft_day")
            }
        else:
            print(f"No match found for {name} in {year}")
    
    return player_map


In [67]:
names = ["Patrick Mahomes", "C.J. Stroud", "Justin Jefferson"]
years = [2017, 2023, 2022]

player_map = get_player_id_map(df["name"], df["draft_year"])
print(player_map)

No match found for Henry Ruggs III in 2020
No match found for Michael Pittman Jr. in 2020
No match found for Laviska Shenault Jr. in 2020
No match found for KJ Hamler in 2020
No match found for AJ Dillon in 2020
No match found for Lynn Bowden Jr. in 2020
No match found for Anthony McFarland Jr. in 2020
No match found for Cole McDonald in 2020
No match found for Kenneth Walker III in 2022
No match found for Velus Jones Jr. in 2022
No match found for Brian Robinson Jr. in 2022
No match found for Calvin Austin III in 2022
No match found for Michael Woods II in 2022
No match found for Grant Dubose in 2023
No match found for Michael Penix in 2024
No match found for Brian Thomas in 2024
No match found for Audric Estime in 2024
No match found for Joe Milton in 2024
{'00-0036442': {'name': 'Joe Burrow', 'rookie_year': 2020, 'position': 'QB', 'team': 'CIN', 'height_in': None, 'weight_lb': None, 'age_on_draft_day': None}, '00-0036212': {'name': 'Tua Tagovailoa', 'rookie_year': 2020, 'position': 

In [68]:
def fill_master_with_player_info(master_df: pd.DataFrame, player_map: dict) -> pd.DataFrame:
    """
    Fill in player_id and other info in master DataFrame from player_map
    """
    master_df = master_df.copy()
    
    # Create DataFrame from player_map
    player_info_df = pd.DataFrame.from_dict(player_map, orient='index')
    player_info_df.reset_index(inplace=True)
    player_info_df.rename(columns={"index": "player_id"}, inplace=True)
    
    # Merge with master on 'name'
    merged_df = master_df.merge(player_info_df, on="name", how="left")
    
    return merged_df


In [69]:
df = fill_master_with_player_info(df, player_map)

In [70]:
# Get player stats for specific season
def get_rookie_stats(player_info_map: dict):
    all_stats = []
    
    for player_id, info in player_info_map.items():
        rookie_year = info["rookie_year"]
        seasonal_stats = nfl.import_seasonal_data([rookie_year])
        
        player_stats = seasonal_stats[seasonal_stats["player_id"] == player_id].copy()
        player_stats["player_name"] = info["name"]
        player_stats["position"] = info["position"]
        player_stats["team"] = info["team"]
        all_stats.append(player_stats)
    
    # Combine all players into a single DataFrame
    return pd.concat(all_stats, ignore_index=True)

In [71]:
rookie_stats = get_rookie_stats(player_map)

print(rookie_stats[:5])

    player_id  season season_type  completions  attempts  passing_yards  \
0  00-0036442    2020         REG          264       404         2688.0   
1  00-0036212    2020         REG          186       290         1814.0   
2  00-0036355    2020         REG          396       595         4336.0   
3  00-0036407    2020         REG            0         0            0.0   
4  00-0036358    2020         REG            0         0            0.0   

   passing_tds  interceptions  sacks  sack_yards  ...    rtd_sh    rfd_sh  \
0           13            5.0   32.0       231.0  ...  0.000000  0.000000   
1           11            5.0   20.0       136.0  ...  0.000000  0.000000   
2           31           10.0   32.0       218.0  ...  0.000000  0.000000   
3            0            0.0    0.0         0.0  ...  0.142857  0.221591   
4            0            0.0    1.0         4.0  ...  0.200000  0.199134   

   rtdfd_sh       dom     w8dom    yptmpa    ppr_sh     player_name  position  \
0  0.

## Fantasy Scoring 
Assumptions for Half-PPR + TE Premium: 
- Passing TD: 4 points
- Passing Yards: 1 point per 25 yards
- Interceptions: -2 points
- Rushing TD: 6 points
- Rushing Yards: 1 point per 10 yards
- Receptions: 0.5 points (Half PPR)
- Receiving TD: 6 points
- Receiving Yards: 1 point per 10 yards
- Fumbles Lost: -2 points
- Tight End Premium: +1 point per reception if the player is a TE

In [26]:
def calculate_halfppr_tep(stats: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate Half-PPR fantasy points with Tight End Premium.

    Args:
        stats: Seasonal stats including player_id, position, receptions, etc.

    Returns:
        pd.DataFrame: Original DataFrame with new 'fantasy_points_halfppr_tep' column
    """
    df = stats.copy()
    
    # Base fantasy points
    df['fantasy_points_halfppr_tep'] = (
        df.get('passing_yards', 0) / 25 +
        df.get('passing_tds', 0) * 4 +
        df.get('interceptions', 0) * -2 +
        df.get('rushing_yards', 0) / 10 +
        df.get('rushing_tds', 0) * 6 +
        df.get('receiving_yards', 0) / 10 +
        df.get('receiving_tds', 0) * 6 +
        df.get('receptions', 0) * 0.5 +
        df.get('rushing_fumbles_lost', 0) * -2 +
        df.get('receiving_fumbles_lost', 0) * -2
    )
    
    # Tight End Premium: +1 point per reception for TEs
    if 'position' in df.columns:
        df.loc[df['position'] == 'TE', 'fantasy_points_halfppr_tep'] += df.loc[df['position'] == 'TE', 'receptions']
    
    return df


In [34]:
rookie_fnt_pts = calculate_halfppr_tep(rookie_stats)
print(rookie_fnt_pts[['player_name', 'position', 'fantasy_points_halfppr_tep']])

        player_name position  fantasy_points_halfppr_tep
0   Patrick Mahomes       QB                       10.36
1  Justin Jefferson       WR                      302.66


## Add Rookie Data to master data frame  

In [ ]:
def add_rookie_stats_to_master(master_df: pd.DataFrame, rookie_stats: pd.DataFrame) -> pd.DataFrame:
    """
    Only adds the following stats:
    Rec, RushYds, RecYds, PassYds, RushTD, RecTD, PassTD, Fumbles, INT

    """
    # Select relevant columns from rookie stats
    rookie_columns_map = {
        'receptions': 'Rec',
        'rushing_yards': 'RushYds',
        'receiving_yards': 'RecYds',
        'passing_yards': 'PassYds',
        'rushing_tds': 'RushTD',
        'receiving_tds': 'RecTD',
        'passing_tds': 'PassTD',
        'rushing_fumbles_lost': 'Fumbles',
        'receiving_fumbles_lost': 'Fumbles',  # Combine fumbles into single column
        'interceptions': 'INT'
    }

    # Copy rookie stats to avoid modifying original
    rookie_copy = rookie_stats.copy()

    # Combine fumbles from rushing and receiving
    rookie_copy['Fumbles'] = rookie_copy.get('rushing_fumbles_lost', 0) + rookie_copy.get('receiving_fumbles_lost', 0)

    # Build new DataFrame with player_id + mapped columns
    rookie_for_merge = pd.DataFrame({'player_id': rookie_copy['player_id']})
    
    for col, new_col in rookie_columns_map.items():
        if col in rookie_copy.columns:
            rookie_for_merge[new_col] = rookie_copy[col]

    # Drop duplicates in case a stat was counted twice (like fumbles)
    rookie_for_merge = rookie_for_merge.groupby('player_id', as_index=False).sum()

    # Merge rookie stats into master df on player_id
    merged_df = master_df.merge(rookie_for_merge, on='player_id', how='left')

    return merged_df


## College Stats: Using CFBD API
https://api.collegefootballdata.com/#/

In [ ]:
# College Football Data API configuration
API_KEY = os.getenv("CFBD_API_KEY")
BASE_URL = "https://api.collegefootballdata.com"
HEADERS = {"Authorization": f"Bearer {API_KEY}"}

In [117]:

# Conference Mapping
team_to_conference = {
    "Air Force": "Mountain West",
    "Akron": "Mid-American",
    "Alabama": "Southeastern",
    "Appalachian State": "Sun Belt",
    "Arizona": "Pac-12",
    "Arizona State": "Pac-12",
    "Arkansas": "Southeastern",
    "Arkansas State": "Sun Belt",
    "Army": "Independent",
    "Auburn": "Southeastern",
    "Ball State": "Mid-American",
    "Baylor": "Big 12",
    "Boise State": "Mountain West",
    "Boston College": "Atlantic Coast",
    "Bowling Green": "Mid-American",
    "Buffalo": "Mid-American",
    "BYU": "Big 12",
    "California": "Pac-12",
    "Central Michigan": "Mid-American",
    "Charlotte": "American",
    "Cincinnati": "Big 12",
    "Clemson": "Atlantic Coast",
    "Coastal Carolina": "Sun Belt",
    "Colorado": "Pac-12",
    "Colorado State": "Mountain West",
    "Duke": "Atlantic Coast",
    "East Carolina": "American",
    "Eastern Michigan": "Mid-American",
    "FIU": "Conference USA",
    "Florida": "Southeastern",
    "Florida Atlantic": "American",
    "Florida International": "Conference USA",
    "Florida State": "Atlantic Coast",
    "Fresno State": "Mountain West",
    "Georgia": "Southeastern",
    "Georgia Southern": "Sun Belt",
    "Georgia State": "Sun Belt",
    "Georgia Tech": "Atlantic Coast",
    "Hawaii": "Mountain West",
    "Houston": "Big 12",
    "Illinois": "Big Ten",
    "Indiana": "Big Ten",
    "Iowa": "Big Ten",
    "Iowa State": "Big 12",
    "Kansas": "Big 12",
    "Kansas State": "Big 12",
    "Kent State": "Mid-American",
    "Kentucky": "Southeastern",
    "Liberty": "Conference USA",
    "Louisiana": "Sun Belt",
    "Louisiana Tech": "Conference USA",
    "Louisville": "Atlantic Coast",
    "LSU": "Southeastern",
    "Marshall": "Conference USA",
    "Maryland": "Big Ten",
    "Memphis": "American",
    "Miami (FL)": "Atlantic Coast",
    "Miami (OH)": "Mid-American",
    "Michigan": "Big Ten",
    "Michigan State": "Big Ten",
    "Minnesota": "Big Ten",
    "Mississippi State": "Southeastern",
    "Missouri": "Southeastern",
    "Navy": "American",
    "Nebraska": "Big Ten",
    "Nevada": "Mountain West",
    "New Mexico": "Mountain West",
    "New Mexico State": "Conference USA",
    "North Carolina": "Atlantic Coast",
    "North Texas": "American",
    "Northern Illinois": "Mid-American",
    "Northwestern": "Big Ten",
    "Notre Dame": "Independent",
    "Ohio": "Mid-American",
    "Ohio State": "Big Ten",
    "Oklahoma": "Big 12",
    "Oklahoma State": "Big 12",
    "Old Dominion": "Sun Belt",
    "Oregon": "Pac-12",
    "Oregon State": "Pac-12",
    "Penn State": "Big Ten",
    "Pittsburgh": "Atlantic Coast",
    "Rice": "American",
    "Rutgers": "Big Ten",
    "San Diego State": "Mountain West",
    "San Jose State": "Mountain West",
    "SMU": "Atlantic Coast",
    "South Alabama": "Sun Belt",
    "South Carolina": "Southeastern",
    "South Florida": "American",
    "Southern Miss": "Sun Belt",
    "Stanford": "Pac-12",
    "Syracuse": "Atlantic Coast",
    "TCU": "Big 12",
    "Temple": "American",
    "Tennessee": "Southeastern",
    "Texas": "Southeastern",
    "Texas A&M": "Southeastern",
    "Texas State": "Sun Belt",
    "Toledo": "Mid-American",
    "UAB": "American",
    "UCF": "Big 12",
    "UCLA": "Pac-12",
    "UNLV": "Mountain West",
    "Utah": "Pac-12",
    "Utah State": "Mountain West",
    "UTSA": "American",
    "Vanderbilt": "Southeastern",
    "Virginia": "Atlantic Coast",
    "Virginia Tech": "Atlantic Coast",
    "Wake Forest": "Atlantic Coast",
    "Washington": "Pac-12",
    "Washington State": "Pac-12",
    "Western Kentucky": "Conference USA",
    "Western Michigan": "Mid-American",
    "Wisconsin": "Big Ten",
    "Wyoming": "Mountain West"
}


In [ ]:
def get_player_season_stats(year: int) -> pd.DataFrame:
    """Pull season-level stats for all players in a given year."""
    url = f"{BASE_URL}/stats/player/season"
    params = {"year": year -1}
    r = requests.get(url, headers=HEADERS, params=params)
    r.raise_for_status()
    df = pd.DataFrame(r.json())
    
    if df.empty:
        return pd.DataFrame()
    
    # Flatten schema
    df = df.rename(columns={"player": "player_name", "playerId": "player_id"})
    # Keep only columns we need
    cols_to_keep = ["season", "player_id", "player_name", "position", "team", "conference","statType", "stat"]
    df = df[cols_to_keep]

    # Pivot statType -> columns
    df = df.pivot_table(
        index=["season", "player_id", "player_name", "position", "team", "conference"],
        columns="statType",
        values="stat",
        aggfunc="sum"
    ).reset_index()

    # Convert stats to numeric
    stat_cols = df.columns.difference(["season", "player_id", "player_name", "position", "team"])
    df[stat_cols] = df[stat_cols].apply(pd.to_numeric, errors="coerce")

    return df

In [96]:
def get_player_usage(year: int) -> pd.DataFrame:
    """Pull player usage data for a given year."""
    url = f"{BASE_URL}/player/usage"
    params = {"year": year}
    r = requests.get(url, headers=HEADERS, params=params)
    r.raise_for_status()
    df = pd.DataFrame(r.json())
    
    if df.empty:
        return pd.DataFrame()

    # Flatten usage dict
    usage_df = pd.json_normalize(df.to_dict(orient="records"), sep="_")
    usage_df = usage_df.rename(columns={"id": "player_id"})
    usage_cols = ["player_id", "season", "usage_overall", "usage_rush", "usage_pass"]
    usage_df = usage_df[usage_cols]

    return usage_df


In [ ]:
def normalize_stats(season_stats, usage_stats, player_list=None):
    """
    Merge season stats with usage and normalize per-snap (usage_overall).
    """
    if player_list is not None:
        season_stats = season_stats[season_stats["player_name"].isin(player_list)]

    # Merge on player_id and season
    df = season_stats.merge(
        usage_stats,
        left_on=["player_id", "season"],
        right_on=["player_id", "season"],
        how="left"
    )

    # Ensure usage exists
    df["usage_overall"] = pd.to_numeric(df["usage_overall"], errors="coerce").fillna(1)

    # Normalize numeric stats only
    stat_cols = df.select_dtypes(include="number").columns.difference(
        ["season", "player_id", "usage_overall"]
    )
    for col in stat_cols:
        df[col + "_norm"] = df[col] / df["usage_overall"]

    return df

In [ ]:
def get_college_stats_for_years(years, player_list=None):
    """
    Fetch college stats and usage data for multiple years, merge them,
    and optionally filter to a list of player names.
    """
    all_stats = []
    for year in years:
        print(f"Fetching college stats for year {year}...")
        season_stats = get_player_season_stats(year)
        season_stats["conference"] = season_stats["team"].map(team_to_conference)

        usage_stats = get_player_usage(year)
        normalized = normalize_stats(season_stats, usage_stats, player_list)
        normalized["college_year"] = year
        all_stats.append(normalized)

    combined_df = pd.concat(all_stats, ignore_index=True)
    return combined_df


# Fetch and normalize college stats
years = [2018, 2019, 2020, 2021, 2022, 2023]
final_df = get_college_stats_for_years(years, player_list=df["name"].tolist())
print(final_df.head())

## Future Improvements
- Include all college seasons with a breakout-season metric
- Weight college stats differently by position for better predictions
- Add combine data (40-yard dash, vertical, etc.) where available

In [ ]:
def get_player_college_profile(player_name: str, draft_year: int):
    """
    Compute final season, career totals, breakout age, and weighted career stats
    for a given player using the CFBD API.

    Args:
        player_name: Full player name (e.g. "Marvin Harrison Jr")
        draft_year: NFL draft year (college final season is draft_year - 1)

    Returns:
        Dict with keys: final_season, career_totals, career_averages,
        breakout_age, weighted_stats. None if player not found.
    """
    url = f"{BASE_URL}/stats/player/season"
    params = {"year": None, "team": None}
    r = requests.get(url, headers=HEADERS, params=params)
    r.raise_for_status()
    df = pd.DataFrame(r.json())

    # Filter for this player
    player_df = df[df["player"].str.lower() == player_name.lower()].copy()
    if player_df.empty:
        print(f"No data found for {player_name}")
        return None

    player_df["stat"] = pd.to_numeric(player_df["stat"], errors="coerce")

    # Pivot so stats are columns
    player_df = player_df.pivot_table(
        index=["season", "player", "position", "team", "conference"],
        columns="statType",
        values="stat",
        aggfunc="sum"
    ).reset_index()

    # Final season stats
    final_season = player_df[player_df["season"] == draft_year - 1]

    # Career totals and averages
    stat_cols = player_df.drop(columns=["season", "player", "position", "team", "conference"])
    career_totals = stat_cols.sum()
    career_averages = career_totals / player_df.shape[0]

    # Breakout age: first season with 500+ receiving yards or 5+ receiving TDs
    breakout_age = None
    if "receivingYards" in player_df.columns and "receivingTDs" in player_df.columns:
        for _, row in player_df.iterrows():
            if row.get("receivingYards", 0) >= 500 or row.get("receivingTDs", 0) >= 5:
                breakout_age = draft_year - row["season"]
                break

    # Weighted stats (recent seasons weighted higher)
    weights = [0.6, 0.3, 0.1]  # Final, second-last, third-last
    seasons_sorted = player_df.sort_values("season", ascending=False).head(3)
    weight_sum = sum(weights[:len(seasons_sorted)])
    weighted_stats = (
        seasons_sorted.drop(columns=["season", "player", "position", "team", "conference"])
        .multiply(weights[:len(seasons_sorted)][::-1], axis=0)
        .sum()
    ) / weight_sum

    return {
        "final_season": final_season,
        "career_totals": career_totals,
        "career_averages": career_averages,
        "breakout_age": breakout_age,
        "weighted_stats": weighted_stats,
    }

In [ ]:
marvin = get_player_college_profile("Marvin Harrison Jr", 2024)

print("Final Season:")
print(marvin["final_season"])

print("\nCareer Totals:")
print(marvin["career_totals"])

print("\nWeighted Career Stats:")
print(marvin["weighted_stats"])

print("\nBreakout Age (years before draft):", marvin["breakout_age"])
